# 🚀 Pipeline Complet : Du Texte Brut à la Prédiction LSTM

Ce notebook combine toutes les techniques apprises :
- Nettoyage de texte (Module 2)
- BOW et TF-IDF (Module 3)
- Word Embeddings (Module 4)
- LSTM pour la classification (Module 5)
- Sauvegarde et réutilisation du modèle

## 📦 1. Import des bibliothèques

In [ ]:
# Imports standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pickle
import json

# NLP et preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Télécharger les ressources NLTK nécessaires
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

print("✅ Toutes les bibliothèques sont importées !")

## 📊 2. Chargement et exploration des données

In [ ]:
# Créons un dataset d'exemple pour la classification de sentiment
# Dans la pratique, vous chargeriez vos propres données

# Exemples de critiques de films (positives et négatives)
texts = [
    "This movie was absolutely fantastic! Best film I've seen all year.",
    "Terrible waste of time. The plot made no sense at all.",
    "Amazing performances by all actors. Highly recommend!",
    "Boring and predictable. I fell asleep halfway through.",
    "A masterpiece of cinema. Every scene was perfectly crafted.",
    "Worst movie ever. Complete disaster from start to finish.",
    "Brilliant storytelling and stunning visuals. Oscar-worthy!",
    "Disappointing sequel. Nothing like the original.",
    "Incredible movie! Had me on the edge of my seat.",
    "Total garbage. Want my money back.",
    "Outstanding film with great character development.",
    "Awful acting and terrible dialogue. Skip this one.",
    "Exceptional movie that touches your heart.",
    "Mediocre at best. Nothing special about it.",
    "Phenomenal! Best movie of the decade.",
    "Horrible experience. Left the theater early."
]

# Labels : 1 = positif, 0 = négatif
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

# Créer un DataFrame
df = pd.DataFrame({
    'text': texts,
    'sentiment': labels
})

print(f"📊 Dataset créé avec {len(df)} exemples")
print(f"\n🔍 Distribution des sentiments:")
print(df['sentiment'].value_counts())
print("\n📝 Exemples de textes:")
df.head()

## 🧹 3. Nettoyage de texte (Module 2)

In [ ]:
# Initialisation des outils
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def clean_text(text, use_lemmatization=True):
    """
    Pipeline de nettoyage complet pour le texte
    """
    # 1. Convertir en minuscules
    text = text.lower()
    
    # 2. Supprimer les URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # 3. Supprimer les mentions et hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # 4. Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 5. Supprimer les chiffres
    text = re.sub(r'\d+', '', text)
    
    # 6. Tokenisation
    tokens = word_tokenize(text)
    
    # 7. Supprimer les stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    # 8. Lemmatization ou Stemming
    if use_lemmatization:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    else:
        tokens = [stemmer.stem(token) for token in tokens]
    
    # 9. Rejoindre les tokens
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

# Appliquer le nettoyage
df['cleaned_text'] = df['text'].apply(clean_text)

print("🧹 Nettoyage terminé !")
print("\n📝 Comparaison avant/après :")
for i in range(3):
    print(f"\nOriginal: {df['text'].iloc[i]}")
    print(f"Nettoyé:  {df['cleaned_text'].iloc[i]}")

## 📊 4. Feature Engineering : BOW et TF-IDF (Module 3)

In [ ]:
# Bag of Words
bow_vectorizer = CountVectorizer(max_features=100)
bow_features = bow_vectorizer.fit_transform(df['cleaned_text']).toarray()

print("🎒 Bag of Words:")
print(f"Forme de la matrice BOW: {bow_features.shape}")
print(f"Vocabulaire (10 premiers mots): {list(bow_vectorizer.vocabulary_.keys())[:10]}")

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=100)
tfidf_features = tfidf_vectorizer.fit_transform(df['cleaned_text']).toarray()

print("\n📈 TF-IDF:")
print(f"Forme de la matrice TF-IDF: {tfidf_features.shape}")

# Visualiser les mots les plus importants selon TF-IDF
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = np.mean(tfidf_features, axis=0)
top_indices = np.argsort(tfidf_scores)[::-1][:10]

plt.figure(figsize=(10, 6))
top_words = [feature_names[i] for i in top_indices]
top_scores = [tfidf_scores[i] for i in top_indices]
plt.barh(top_words, top_scores)
plt.xlabel('Score TF-IDF moyen')
plt.title('Top 10 mots selon TF-IDF')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 🧠 5. Préparation pour LSTM avec Embeddings (Module 4 & 5)

In [ ]:
# Paramètres pour le modèle LSTM
MAX_WORDS = 1000  # Taille du vocabulaire
MAX_SEQUENCE_LENGTH = 100  # Longueur maximale des séquences
EMBEDDING_DIM = 100  # Dimension des embeddings

# Tokenizer de Keras pour préparer les séquences
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned_text'])

# Convertir les textes en séquences d'indices
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# Padding des séquences
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
y = np.array(df['sentiment'])

print(f"🔢 Taille du vocabulaire: {len(tokenizer.word_index)}")
print(f"📏 Forme des données X: {X.shape}")
print(f"\n📝 Exemple de conversion:")
print(f"Texte original: {df['cleaned_text'].iloc[0]}")
print(f"Séquence: {sequences[0]}")
print(f"Après padding: {X[0][:20]}...")

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🚀 6. Construction du modèle LSTM

In [ ]:
# Construction du modèle LSTM
model = Sequential([
    # Couche d'embedding
    Embedding(input_dim=MAX_WORDS, 
              output_dim=EMBEDDING_DIM,
              name='embedding_layer'),
    
    # LSTM bidirectionnel pour capturer le contexte dans les deux sens
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)),
    
    # Deuxième couche LSTM
    LSTM(32, dropout=0.3, recurrent_dropout=0.3),
    
    # Couches denses
    Dense(32, activation='relu'),
    Dropout(0.5),
    
    # Couche de sortie
    Dense(1, activation='sigmoid')
])

# Compilation
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Construire le modèle avec la forme des données
model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

# Afficher l'architecture
model.summary()

# Visualisation de l'architecture (après construction)
try:
    tf.keras.utils.plot_model(model, to_file='lstm_model.png', show_shapes=True, show_layer_names=True)
    print("✅ Diagramme du modèle sauvegardé dans 'lstm_model.png'")
except Exception as e:
    print(f"⚠️ Impossible de créer le diagramme: {e}")
    print("Le modèle est quand même prêt pour l'entraînement !")

## 📈 7. Entraînement du modèle

In [ ]:
# Callbacks pour l'optimisation
callbacks = [
    # Early stopping pour éviter l'overfitting
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Réduction du learning rate
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Sauvegarde du meilleur modèle
    ModelCheckpoint(
        'best_lstm_model.h5',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

# Entraînement
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

# Visualisation de l'entraînement
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Evolution de la Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Evolution de l\'Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 📊 8. Évaluation du modèle

In [ ]:
# Prédictions sur le test set
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Métriques
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Accuracy sur le test set: {accuracy:.2%}")

# Rapport de classification
print("\n📊 Rapport de classification:")
print(classification_report(y_test, y_pred, target_names=['Négatif', 'Positif']))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Négatif', 'Positif'],
            yticklabels=['Négatif', 'Positif'])
plt.title('Matrice de Confusion')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()

## 💾 9. Sauvegarde du modèle et des préprocesseurs

In [ ]:
# 1. Sauvegarder le modèle Keras
model.save('sentiment_lstm_model.h5')
print("✅ Modèle LSTM sauvegardé dans 'sentiment_lstm_model.h5'")

# 2. Sauvegarder le tokenizer
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)
print("✅ Tokenizer sauvegardé dans 'tokenizer.json'")

# 3. Sauvegarder les paramètres du modèle
model_config = {
    'max_words': MAX_WORDS,
    'max_sequence_length': MAX_SEQUENCE_LENGTH,
    'embedding_dim': EMBEDDING_DIM,
    'model_type': 'LSTM_Bidirectional',
    'task': 'sentiment_analysis'
}

with open('model_config.json', 'w') as f:
    json.dump(model_config, f, indent=4)
print("✅ Configuration sauvegardée dans 'model_config.json'")

# 4. Sauvegarder aussi les vectorizers (optionnel)
with open('bow_vectorizer.pkl', 'wb') as f:
    pickle.dump(bow_vectorizer, f)
    
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)
    
print("✅ Vectorizers BOW et TF-IDF sauvegardés")

## 🔮 10. Fonction de prédiction pour nouveaux textes

In [ ]:
class SentimentPredictor:
    def __init__(self, model_path, tokenizer_path, config_path):
        """Charger le modèle et les préprocesseurs"""
        # Charger le modèle
        self.model = load_model(model_path)
        
        # Charger le tokenizer
        with open(tokenizer_path, 'r', encoding='utf-8') as f:
            tokenizer_json = f.read()
        self.tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenizer_json)
        
        # Charger la configuration
        with open(config_path, 'r') as f:
            self.config = json.load(f)
            
        # Initialiser les outils de nettoyage
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def clean_text(self, text):
        """Nettoyer le texte"""
        text = text.lower()
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\d+', '', text)
        
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words]
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        return ' '.join(tokens)
    
    def predict(self, text):
        """Prédire le sentiment d'un texte"""
        # Nettoyer le texte
        cleaned = self.clean_text(text)
        
        # Convertir en séquence
        sequence = self.tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(sequence, 
                             maxlen=self.config['max_sequence_length'],
                             padding='post',
                             truncating='post')
        
        # Prédiction
        prediction = self.model.predict(padded, verbose=0)[0][0]
        
        # Interprétation
        sentiment = 'Positif' if prediction > 0.5 else 'Négatif'
        confidence = prediction if prediction > 0.5 else 1 - prediction
        
        return {
            'text': text,
            'cleaned_text': cleaned,
            'sentiment': sentiment,
            'confidence': float(confidence),
            'raw_score': float(prediction)
        }

# Instancier le prédicteur
predictor = SentimentPredictor(
    model_path='sentiment_lstm_model.h5',
    tokenizer_path='tokenizer.json',
    config_path='model_config.json'
)

print("✅ Prédicteur chargé et prêt !")

## 🎯 11. Test sur de nouveaux textes

In [ ]:
# Textes de test
test_texts = [
    "This product is amazing! I love it so much.",
    "Terrible quality. Complete waste of money.",
    "Not bad, but could be better. Average product.",
    "Absolutely fantastic! Exceeded all my expectations!",
    "Disappointing. Does not work as advertised."
]

print("🔮 Prédictions sur de nouveaux textes:\n")

for text in test_texts:
    result = predictor.predict(text)
    print(f"📝 Texte: '{text}'")
    print(f"🧹 Nettoyé: '{result['cleaned_text']}'")
    print(f"😊 Sentiment: {result['sentiment']} (Confiance: {result['confidence']:.2%})")
    print(f"📊 Score brut: {result['raw_score']:.3f}")
    print("-" * 80)

## 💻 12. Interface interactive pour tester

In [ ]:
def analyse_sentiment_interactif():
    """Interface interactive pour tester le modèle"""
    print("=" * 80)
    print("🤖 Analyseur de Sentiment LSTM")
    print("=" * 80)
    print("Entrez du texte pour analyser le sentiment (ou 'quit' pour arrêter)\n")
    
    while True:
        text = input("\n💬 Votre texte: ")
        
        if text.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Au revoir !")
            break
        
        if text.strip():
            result = predictor.predict(text)
            
            # Affichage avec emojis selon le sentiment
            emoji = "😊" if result['sentiment'] == 'Positif' else "😢"
            
            print(f"\n{emoji} Sentiment: {result['sentiment']}")
            print(f"📊 Confiance: {result['confidence']:.2%}")
            
            # Barre de progression visuelle
            bar_length = 30
            filled = int(bar_length * result['raw_score'])
            bar = "█" * filled + "░" * (bar_length - filled)
            print(f"\nNégatif [{bar}] Positif")
            print(f"Score: {result['raw_score']:.3f}")

# Décommenter pour utiliser l'interface interactive
# analyse_sentiment_interactif()

## 📦 13. Script Python standalone pour utilisation future

In [ ]:
# Créer un script Python réutilisable
script_content = '''#!/usr/bin/env python3
"""
Script d'analyse de sentiment avec LSTM
Utilisation: python sentiment_analyzer.py "Votre texte ici"
"""

import sys
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import string

class SentimentAnalyzer:
    def __init__(self):
        # Charger le modèle
        self.model = load_model('sentiment_lstm_model.h5')
        
        # Charger le tokenizer
        with open('tokenizer.json', 'r', encoding='utf-8') as f:
            tokenizer_json = f.read()
        self.tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenizer_json)
        
        # Charger la config
        with open('model_config.json', 'r') as f:
            self.config = json.load(f)
            
        # Outils NLP
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
    
    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r'https?://\\S+|www\\.\\S+', '', text)
        text = re.sub(r'@\\w+|#\\w+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\\d+', '', text)
        
        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words]
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        return ' '.join(tokens)
    
    def analyze(self, text):
        cleaned = self.clean_text(text)
        sequence = self.tokenizer.texts_to_sequences([cleaned])
        padded = pad_sequences(sequence, 
                             maxlen=self.config['max_sequence_length'],
                             padding='post')
        
        prediction = self.model.predict(padded, verbose=0)[0][0]
        sentiment = 'Positif' if prediction > 0.5 else 'Négatif'
        confidence = prediction if prediction > 0.5 else 1 - prediction
        
        return sentiment, confidence, prediction

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Usage: python sentiment_analyzer.py \"Votre texte\"")
        sys.exit(1)
    
    text = ' '.join(sys.argv[1:])
    
    analyzer = SentimentAnalyzer()
    sentiment, confidence, score = analyzer.analyze(text)
    
    emoji = "😊" if sentiment == 'Positif' else "😢"
    print(f"\n{emoji} Sentiment: {sentiment}")
    print(f"📊 Confiance: {confidence:.2%}")
    print(f"📈 Score: {score:.3f}")
'''

# Sauvegarder le script
with open('sentiment_analyzer.py', 'w') as f:
    f.write(script_content)

print("✅ Script standalone créé : 'sentiment_analyzer.py'")
print("\n📌 Utilisation:")
print("   python sentiment_analyzer.py \"Votre texte à analyser\"")

## 🎉 Conclusion

Félicitations ! Vous avez créé un pipeline complet de NLP qui combine :

1. **Nettoyage de texte** (Module 2)
2. **Feature engineering** avec BOW et TF-IDF (Module 3)
3. **Word embeddings** (Module 4)
4. **LSTM** pour la classification (Module 5)

### 📁 Fichiers sauvegardés :
- `sentiment_lstm_model.h5` : Le modèle LSTM entraîné
- `tokenizer.json` : Le tokenizer pour préparer les textes
- `model_config.json` : La configuration du modèle
- `sentiment_analyzer.py` : Script Python standalone
- `bow_vectorizer.pkl` et `tfidf_vectorizer.pkl` : Vectorizers (optionnel)

### 🚀 Prochaines étapes :
- Entraîner sur un dataset plus large
- Essayer d'autres architectures (GRU, Transformers)
- Déployer le modèle dans une API
- Améliorer le preprocessing
- Ajouter plus de classes (sentiment neutre, etc.)